In [1]:
import json

# Specify the path to your JSON file
json_file_path = '/Users/vasu/Desktop/project/chatbot_ic_NLP/lib/NLP-Models/2013pubmed.json'

# Open the JSON file and load the data
with open(json_file_path, 'r') as json_file:
    dataset = json.load(json_file)

# Print the loaded dataset
print(len(dataset))

1620


In [2]:
%pip install -Uq chromadb numpy datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
import chromadb

from chromadb import EmbeddingFunction


client = chromadb.Client()

In [3]:
from chromadb.utils import embedding_functions

In [4]:

from sentence_transformers import SentenceTransformer
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1")
my_collection = client.create_collection("2013pubmed", embedding_function=sentence_transformer_ef)chromadb start


In [5]:
# Extract data from the dataset and store it in the collection
my_collection.add(
    ids=[str(entry['PMID']) for entry in dataset],
    documents=[entry['Abstract'] for entry in dataset],
    metadatas=[
        {'title': entry['Title'], 'author': entry['Author']} for entry in dataset
    ],
)



In [9]:
# Choose a specific document (e.g., using its PMID)
pmid_to_inspect = '25529585'

retrieved_data = my_collection.get(ids=[pmid_to_inspect])

# Print the retrieved data
print(retrieved_data)

{'ids': ['25529585'], 'embeddings': None, 'metadatas': [{'author': 'Roman Marcus Carrasco', 'title': 'Apparently abnormal Wechsler Memory Scale index score patterns in the normal population'}], 'documents': ['Interpretation of the Wechsler Memory Scale-Fourth Edition may involve examination of multiple memory index score contrasts and similar comparisons with Wechsler Adult Intelligence Scale-Fourth Edition ability indexes. Standardization sample data suggest that 15-point differences between any specific pair of index scores are relatively uncommon in normal individuals, but these base rates refer to a comparison between a single pair of indexes rather than multiple simultaneous comparisons among indexes. This study provides normative data for the occurrence of multiple index score differences calculated by using Monte Carlo simulations and validated against standardization data. Differences of 15 points between any two memory indexes or between memory and ability indexes occurred in 

In [10]:
print(my_collection.get(ids=['21723361'], include= ['documents', 'embeddings']))


{'ids': ['21723361'], 'embeddings': [[-0.023051582276821136, -0.015295928344130516, 0.04858016222715378, -0.03580956906080246, 0.027030054479837418, 0.018105603754520416, 0.013021391816437244, 0.03626439720392227, -0.06448755413293839, -0.026079263538122177, 0.05672258511185646, -0.03270727023482323, -0.11501924693584442, 0.010153699666261673, -0.01929684728384018, 0.05824482440948486, 0.039322350174188614, 0.008022190071642399, 0.026757463812828064, 0.04715750738978386, -0.02840557135641575, 0.025222819298505783, 0.061552178114652634, -0.0051207453943789005, -0.05223706364631653, 0.09982550889253616, -0.022406315430998802, 0.0068510896526277065, 0.05093911662697792, 0.050796907395124435, 0.04798431694507599, 0.09668426215648651, 0.10195185244083405, -0.03955492004752159, -0.08661452680826187, -0.04658474773168564, 0.0017812338192015886, -0.03060678392648697, -0.057935502380132675, 0.013549855910241604, 0.04589097946882248, 0.013747917488217354, -0.045720554888248444, 0.075959235429763

In [11]:
from sentence_transformers import SentenceTransformer, util
# User's question
user_question = 'CASK Disorders'

# Embed the user's question
user_question_embedding = sentence_transformer_ef([user_question])[0]

search_results = my_collection.query(query_embeddings=[user_question_embedding], n_results=5)
print(user_question)
# Print the search results
for i, result in enumerate(search_results['ids'][0]):
    document_id = result
    metadata = search_results['metadatas'][0][i]  # Access the corresponding metadata
    similarity_score = search_results['distances'][0][i]  # Access the corresponding similarity score
    document = search_results['documents'][0][i]  # Access the corresponding document

    print(f"PMID: {document_id}")
    print(f"Title: {metadata['title']}")
    print(f"Author: {metadata['author']}")
    print(document)
    print(f"Similarity Score: {similarity_score}")
    print("---------------")


CASK Disorders
PMID: 24278995
Title: CASK Disorders
Author: Ute Moog
Clinical characteristics: CASK disorders include a spectrum of phenotypes in both females and males. Two main types of clinical presentation are seen: Microcephaly with pontine and cerebellar hypoplasia (MICPCH), generally associated with pathogenic loss-of-function variants in CASKX-linked intellectual disability (XLID) with or without nystagmus, generally associated with hypomorphic CASK pathogenic variants MICPCH is typically seen in females with moderate-to-severe intellectual disability, progressive microcephaly with or without ophthalmologic anomalies, and sensorineural hearing loss. Most are able to sit independently; 20%-25% attain the ability to walk; language is nearly absent in most. Neurologic features may include axial hypotonia, hypertonia/spasticity of the extremities, and dystonia or other movement disorders. Nearly 40% have seizures by age ten years. Behaviors may include sleep disturbances, hand ster

In [15]:
!which chromadb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
chromadb not found
